# Clasificación de mecanismos de ausencia

En este capítulo se estudia el tipo de mecanismo que origina los valores faltantes en la base de datos.  
Existen diferentes categorías (MCAR, MAR, MNAR) que describen cómo y por qué ocurren las ausencias.

**Puntos clave:**
- Revisar patrones de ausencia en relación con otras variables.
- Analizar correlaciones para detectar dependencia entre los faltantes y las variables observadas.
- Determinar si los datos faltan al azar o bajo un mecanismo no aleatorio.

Esta clasificación orienta la elección de la técnica de imputación más apropiada y asegura que los resultados posteriores sean válidos.



In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

# Subir un nivel desde la carpeta "secciones"
csv_path = Path.cwd().parent / "base_imputacion_mixta_1000.csv"

df = pd.read_csv(csv_path)

print("Dimensiones:", df.shape)
df.head()


Dimensiones: (1000, 12)


,fecha,sexo,ciudad,nivel_educativo,segmento,estado_civil,edad,altura_cm,ingresos,gasto_mensual,puntuacion_credito,demanda
0,2024-01-01,F,Medellín,NaN,B,Unión libre,19.0,161.821754,3574.753806,1832.731832,640.465372,119.202995
1,2024-01-02,F,Barranquilla,NaN,B,NaN,52.0,167.819566,3163.626815,NaN,533.108430,124.457874
2,2024-01-03,M,Bogotá,Secundaria,B,Soltero/a,38.0,165.756219,2765.672259,1219.535074,491.016910,NaN
3,2024-01-04,F,Bogotá,NaN,B,Casado/a,57.0,160.642670,4320.397345,1908.324816,NaN,129.426792
4,2024-01-05,M,Cali,Técnico,B,Soltero/a,67.0,151.402909,NaN,1887.385697,610.213994,133.916319


### Descripción del dataset

El dataset contiene **1000 filas y 12 columnas**, con información de tipo socio-demográfico, económico y de consumo.  
Las variables principales son:

- **fecha**: Día de registro (formato `YYYY-MM-DD`).
- **sexo**: Género del individuo (F/M).
- **ciudad**: Ciudad de residencia (ej. Medellín, Barranquilla, Bogotá, Cali).
- **nivel_educativo**: Nivel educativo alcanzado (Secundaria, Técnico, etc.). Contiene valores faltantes.
- **segmento**: Segmento de clasificación (ejemplo: "B").
- **estado_civil**: Estado civil del individuo (Unión libre, Soltero/a, Casado/a). Contiene valores faltantes.
- **edad**: Edad en años.
- **altura_cm**: Altura en centímetros.
- **ingresos**: Ingresos mensuales (algunas filas con valores faltantes).
- **gasto_mensual**: Gasto mensual reportado (con algunos valores faltantes).
- **puntuacion_credito**: Puntuación crediticia (con valores nulos).
- **demanda**: Valor numérico asociado a la demanda estimada de un producto/servicio.

#### Observaciones iniciales
- Hay **valores faltantes** en varias columnas: `nivel_educativo`, `estado_civil`, `ingresos`, `gasto_mensual`, `puntuacion_credito` y `demanda`.
- Las variables combinan **datos categóricos** (sexo, ciudad, nivel educativo, estado civil, segmento) y **datos numéricos** (edad, altura, ingresos, gasto, puntuación de crédito, demanda).
- El dataset parece adecuado para análisis de segmentación, predicción de demanda o análisis de comportamiento de consumo.


In [5]:
import numpy as np
import pandas as pd
from scipy.stats import pointbiserialr, ttest_ind

resumen = []

num_cols_all = df.select_dtypes(include=[np.number]).columns.tolist()

for col in df.columns:
    # máscara binaria de faltantes de la variable 'col'
    mask = df[col].isna().astype(int)

    # si la máscara no tiene variación (todo 0 o todo 1), no hay nada que asociar
    if mask.nunique() <= 1:
        resumen.append({
            "variable": col,
            "max_corr_mask_vs_numeric": np.nan,
            "pval_diff_mean_example": np.nan,
            "nota": "sin variación en la máscara"
        })
        continue

    # columnas numéricas candidatas (excluye 'col' si también es numérica)
    num_cols = [c for c in num_cols_all if c != col]

    max_r = np.nan  # |correlación| máxima entre máscara y numéricas
    for nc in num_cols:
        x = df[nc].to_numpy(dtype=float)
        m = mask.to_numpy()

        # descarta filas con NaN en x
        valid = ~np.isnan(x)
        x = x[valid]
        m2 = m[valid]

        # si no hay variación o no hay datos suficientes, saltar
        if x.size < 8 or np.nanstd(x) == 0 or np.unique(m2).size < 2:
            continue

        # correlación punto-biserial (binaria vs numérica)
        r, p = pointbiserialr(m2, x)
        if not np.isnan(r):
            max_r = np.nanmax([max_r, abs(r)]) if not np.isnan(max_r) else abs(r)

    # t-test simple como señal adicional (elige una numérica representativa)
    pval = np.nan
    if num_cols:
        nc = num_cols[0]
        g1 = df.loc[mask == 1, nc].dropna()
        g0 = df.loc[mask == 0, nc].dropna()
        if len(g1) > 5 and len(g0) > 5 and g1.std(ddof=0) > 0 and g0.std(ddof=0) > 0:
            _, pval = ttest_ind(g1, g0, equal_var=False, nan_policy="omit")

    resumen.append({
        "variable": col,
        "max_corr_mask_vs_numeric": max_r,
        "pval_diff_mean_example": pval,
        "nota": None if not np.isnan(max_r) else "sin numéricas válidas"
    })

clasif = pd.DataFrame(resumen).sort_values("max_corr_mask_vs_numeric", ascending=False)
clasif.head(15)



,variable,max_corr_mask_vs_numeric,pval_diff_mean_example,nota
9,gasto_mensual,0.177659,0.002410,None
2,ciudad,0.111821,0.106080,None
8,ingresos,0.108755,0.109780,None
11,demanda,0.087479,0.131425,None
3,nivel_educativo,0.077216,0.586320,None
7,altura_cm,0.072031,0.289901,None
4,segmento,0.061822,0.060034,None
1,sexo,0.057228,0.189338,None
5,estado_civil,0.050197,0.458526,None
10,puntuacion_credito,0.030588,0.841998,None


### Comentario del dataset

El dataset contiene **1000 registros y 12 variables**. Incluye datos **categóricos** (sexo, ciudad, nivel_educativo, segmento, estado_civil) y **numéricos** (edad, altura_cm, ingresos, gasto_mensual, puntuacion_credito, demanda).

- **fecha**: registro diario (`YYYY-MM-DD`).  
- **sexo / ciudad / nivel_educativo / estado_civil / segmento**: información demográfica y de clasificación, con valores faltantes en educación y estado civil.  
- **edad / altura_cm**: características físicas básicas.  
- **ingresos / gasto_mensual / puntuacion_credito / demanda**: variables económicas y de comportamiento, varias con valores nulos.  

**Observación:** existen datos faltantes en variables clave, lo que requiere imputación o tratamiento previo. La mezcla de información demográfica, económica y de consumo lo hace adecuado para **segmentación de clientes** y **modelos predictivos de demanda**.
